In [7]:
from katonic.ml import MyClient
import os
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss, recall_score, f1_score, precision_score
from sklearn.model_selection import train_test_split
import pandas as pd

In [12]:
myclient = MyClient()
mlflow = myclient.mlflow
client = myclient.client
mlflow.create_experiment("test_exp")

'32'

In [9]:
def metric(actual, pred):
    acc_score = accuracy_score(actual, pred)
    recall = recall_score(actual, pred)
    precision_scr = precision_score(actual, pred)
    f1_scr = f1_score(actual, pred)
    auc_roc = roc_auc_score(actual, pred)
    log_los = log_loss(actual, pred)

    return (
        acc_score,
        auc_roc,
        log_los,
        recall,
        f1_scr,
        precision_scr
    )

In [10]:
#mlflow.xgboost.autolog()
exp_name = "mlflow-test-XGB2"
mlflow.set_experiment(exp_name)
exp_details = mlflow.get_experiment_by_name(exp_name)
with mlflow.start_run(run_name=exp_name):
    df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv')
    x = df.drop(columns=['Outcome'], axis=1)
    y = df['Outcome']
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.20, random_state=98)
    # The predicted column is "quality" which is a scalar from [3, 9]
    model_clf = XGBClassifier(random_state=0)
    #mlflow.log_param("")
    model_clf.fit(X_train, y_train)
    
    y_pred = model_clf.predict(X_test)
    (acc_score, auc_roc, log_los, recall, f1_scr, precision_scr) = metric(y_test, y_pred)

    model_metrics = {
        "accuracy_score": acc_score,
        "roc_auc_score": auc_roc,
        "log_loss": log_los,
        "recall": recall,
        "f1_score": f1_scr,
        "precision_score": precision_scr
    }

    for metric_name, score in model_metrics.items():
        mlflow.log_metric(metric_name, score)

    model_info = mlflow.xgboost.log_model(xgb_model=model_clf, artifact_path="model")
    

2023/10/24 13:34:31 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-test-XGB2' does not exist. Creating a new experiment.
2023/10/24 13:34:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/lib/python3.9/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/10/24 13:34:32 WARNI

In [11]:
# Note that the xgboost model flavor only supports an instance of xgboost.Booster, not models that implement the scikit-learn API.